# **MDD diagnosis based on EEG feature fusion and improved feature selection**
 Wan Chen, Yanping Cai*, Aihua Li, Yanzhao Su, Ke Jiang
 Rocket Force University of Engineering, Xi’an 710025, China

## **一、包导入**

In [10]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
import mne
import matplotlib.pyplot as plt
import os
import utilities as ut
import mne_connectivity

In [11]:
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号
plt.rcParams['text.usetex'] = True
np.set_printoptions(threshold=np.inf)# 设置NumPy的打印选项，以打印整个数组

## **二、文件批量读取读取->数据预处理->频带划分->求各频带功能连接矩阵**

### (0) 电极重命名
### (1) 数据滤波: 50Hz陷波滤波,0.5-50Hz带通滤波    
    凹陷滤波，指的是在某个频率范围内的信号会被衰减过滤掉，而这个频率范围以外的信号会被保留下来。
    这个操作通常是用来去除50Hz市电的干扰。我国的50Hz，因此在收集到的信号中，会有一个非常强烈的50Hz频段的信号存在，这就可以用凹陷滤波来去掉。
### (2) 伪迹去除：使用独立成分分析（ICA）
    由于ICA对低频分离效果不好,这里对高通1Hz的数据进行ICA及相关成分剔除，再应用到高通0.1Hz的数据上
### (3)重参考:使用全脑平均参考
    首先要解释的是参考的定义。我们看到采集之后的数据是一个个的数值，但这个数值是什么意思呢，就是电极所在位置跟参考电极之间的电位差。
    一般在脑电记录的时候会采用的参考电极有鼻尖参考，cz或头顶中央参考，还有单侧乳突参考，乳突就是耳朵后面一小块突起的区域。我们所看到的每个通道的数值，其实就是指这个通道跟参考通道之间的电位差。
    在分析数据的时候，有时候我们会想要转换参考点的位置。因为不同位置的参考，会对数据造成一定的影响。比如记录时采用的是cz或头顶中央参考，那么自然地，距离cz点较近的电极点，记录到的电位差会非常小，而离得远的电极记录到的电位差就自然会大一点，这种大与小的差异，并不是由认知活动产生的，而是由记录方式产生的。又比如，单侧乳突参考，那脑袋左边的电极点跟右边的电极点，也会存在着记录方式不同产生的电位差不同。
    常用的一些参考位置有双侧乳突平均参考，指将两个乳突数据的平均值作为参考数据，或者是全脑平均参考，指的是将全脑所有数据的均值作为参考数据的方法，不过使用全脑平均参考的时候要注意，眼电数据不要纳入其中，因为眼电数据的波动起伏非常大，很容易对数据造成比较大的干扰。
### (4)数据分段：提取60秒后的100秒数据，分割成10秒的时间段
### (5)频带提取：Delta,Theta,Alpha,Beta,Gamma,Full

In [12]:
raw_data_dir = './data/'

all_files = []
for _dir in os.listdir(raw_data_dir):
    sub_dir = os.path.join(raw_data_dir, _dir)
    if os.path.isdir(sub_dir):
        for file in os.listdir(sub_dir):
            edf_file_path = os.path.join(sub_dir, file)
            all_files = all_files + [edf_file_path]
display = 1

# num_eeg_data = 19 * 256 * 10 # 19通道，256HZ * 10s
# num_conns_data = 19 * (19-1) // 2 * len(ut.bands) * len(ut.conn_methods) # 下三角矩阵元素数量19*(19-1)/2, 频带数，特性数
# num_eeg_files = 58
# entire_fusion_features_matrix = np.zeros(shape=(num_eeg_data * num_conns_data * num_eeg_files), dtype=float)
for edf_file_path in all_files:
    raw = mne.io.read_raw_edf(edf_file_path, preload=True)

    ### (0) 电极重命名
    # valid channel names
    valid_channel_names = ['EEG Fp1-LE', 'EEG F3-LE', 'EEG C3-LE', 'EEG P3-LE', 'EEG O1-LE', 'EEG F7-LE', 'EEG T3-LE', 'EEG T5-LE', 'EEG Fz-LE', 'EEG Fp2-LE', 'EEG F4-LE', 'EEG C4-LE', 'EEG P4-LE', 'EEG O2-LE', 'EEG F8-LE', 'EEG T4-LE', 'EEG T6-LE', 'EEG Cz-LE', 'EEG Pz-LE']
    raw.pick(valid_channel_names)
    # 创建电极名称映射字典
    channel_mapping = {
        'EEG Fp1-LE':'Fp1', 'EEG F3-LE':'F3',   'EEG C3-LE':'C3',   'EEG P3-LE':'P3',   'EEG O1-LE':'O1',
        'EEG F7-LE':'F7',   'EEG T3-LE':'T3',   'EEG T5-LE':'T5',   'EEG Fz-LE':'Fz',   'EEG Fp2-LE':'Fp2',
        'EEG F4-LE':'F4',   'EEG C4-LE':'C4',   'EEG P4-LE':'P4',   'EEG O2-LE':'O2',   'EEG F8-LE':'F8',
        'EEG T4-LE':'T4',   'EEG T6-LE':'T6',   'EEG Cz-LE':'Cz',   'EEG Pz-LE':'Pz',
        # 'EEG A2-A1': 'A2','EEG 23A-23R':'23A','EEG 24A-24R': '24A',
    }
    raw.rename_channels(channel_mapping)# 电极重命名为标准名称
    montage = mne.channels.make_standard_montage('standard_1020')# 创建标准的10-20系统电极布局
    raw.set_montage(montage)# 应用电极布局到数据
    # raw.plot_sensors(ch_type='eeg', show_names=True)

    ### (1) 数据滤波
    raw.notch_filter(freqs=50) # 50Hz陷波滤波    
    raw.filter(l_freq=0.5, h_freq=50, method='iir') # 0.5-50Hz带通滤波

    # (2) 伪迹去除：使用独立成分分析（ICA）
    ica = mne.preprocessing.ICA(max_iter='auto', random_state=97)  
    raw_for_ica = raw.copy().filter(l_freq=1, h_freq=None)
    ica.fit(raw_for_ica)
    file_name_witout_extension = os.path.splitext(os.path.basename(edf_file_path))[0]
    ica.exclude = ut.excludes[file_name_witout_extension] # 选择伪迹成分进行去除
    raw_clean = ica.apply(raw_for_ica.copy())

    # (3) 重参考：使用全脑平均参考
    raw_clean.set_eeg_reference('average', projection=False)
    # (4) 数据分割
    raw_clean_temp = raw_clean.copy()
    raw_clean_segment = raw_clean_temp.crop(tmin=60, tmax=160) # 提取60秒后的100秒数据    
    epochs = mne.make_fixed_length_epochs(raw_clean_segment, duration=10, overlap=0) # 将数据分割成10秒的时间段
    # (5) 频带提取
    epochs_spectrum = epochs.compute_psd(fmin=0.5,fmax=50, method='welch')
    if display:
        epochs_spectrum.plot_topomap(bands=ut.bands) # ,vlim='joint'
        %matplotlib inline
        plt.figure()
        ax = plt.axes()
        epochs_spectrum.plot(average=True, color='blue', axes=ax)
        # psds, freqs = epochs_spectrum.get_data(return_freqs=True)
        # # 提取每个频带的功率
        # band_powers = {}
        # for band_name, (fmin, fmax) in bands.items():
        #     band_powers[band_name] = np.mean(psds[:, (freqs >= fmin) & (freqs <= fmax)], axis=1)

        # # 打印每个频带的功率
        # for band_name, power in band_powers.items():
        #     print(f"Mean power in {band_name} band: {np.mean(power)}")
    
    # (6) 计算各频带功能连接矩阵
    # epochs_data = epochs.get_data()
    # for epoch_idx in range(epochs_data.shape[0]):
    bands_conns = {}
    for band_name, (fmin, fmax) in ut.bands.items():
        conns = mne_connectivity.spectral_connectivity_epochs(epochs, method=ut.conn_methods, mode='multitaper', sfreq=epochs.info['sfreq'], fmin=fmin, fmax=fmax, faverage=True, indices=None)
        conns_dict = {}
        for i in range(len(ut.conn_methods)):
            conn_matrix = conns[i].get_data().reshape(19,19)
            conn_matrix = np.sqrt(np.real(conn_matrix) **2, np.imag(conn_matrix) ** 2)
            conns_dict[ut.conn_methods[i]] = conn_matrix
        bands_conns[band_name] = conns_dict
    if  display:
        rows = len(ut.bands)
        cols = len(ut.conn_methods)
        plt.figure(figsize=(15, 15))
        plt_idx = 1
        for band_name, _coons in bands_conns.items():
            for conn_method, conn_matrix in _coons.items():
                plt.subplot(rows, cols, plt_idx)
                plt_idx += 1
                # 可视化连接矩阵
                plt.imshow(conn_matrix, cmap='coolwarm', interpolation='nearest')
                plt.colorbar()
                plt.xticks(np.arange(0, 19, 1))  # 设置x轴的刻度从0到10，步长为1
                plt.yticks(np.arange(0, 19, 1))  # 设置y轴的刻度从-1到1，步长为1
                plt.title(f'{band_name} {conn_method}')
                plt.axis('off')  # 关闭坐标轴
        plt.show()
        display=0
    # (7) 取功能连接矩阵的下三角形 19*(19-1)/2=171个值为单个频带单个功能的特征向量
    conn_eigenvectors_dict={} #单个功能链接矩阵的维度为171*6=1026  key:conn_method,value:ndarray(shape=(1026))
    for conn_method in ut.conn_methods:
        conn_eigenvector = None
        for band_name in bands_conns:
            conn_matrix = bands_conns[band_name][conn_method]
            row_idx, col_idx = np.tril_indices(19, k=-1)  # k=-1 表示不包含对角线
            conn_eigenvector = conn_matrix[row_idx, col_idx] if conn_eigenvector is None else np.hstack((conn_eigenvector, conn_matrix[row_idx, col_idx]))
            # print(conn_eigenvector.shape)
        conn_eigenvectors_dict[conn_method] = conn_eigenvector
    conn_eigenvectors = None
    for band_name, curr_eigenvectors in conn_eigenvectors_dict.items():
        conn_eigenvectors = conn_eigenvectors_dict[band_name] if conn_eigenvectors is None else np.hstack((conn_eigenvectors, conn_eigenvectors_dict[band_name]))
    output_filename = "H_conn_eigenvectors.txt" if file_name_witout_extension.startswith("H") else "MDD_conn_eigenvectors.txt"
    with open(output_filename, "a") as f:
        print(conn_eigenvectors, file = f)
    # entire_fusion_features_matrix[i * num_eeg_data:i * num_eeg_data + num_conns_data] = conn_eigenvectors

Extracting EDF parameters from f:\脑电\EEG论文\论文复现\data\H\H S1 EO.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 89855  =      0.000 ...   350.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-stop filter from 49 - 51 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 49.38
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 49.12 Hz)
- Upper passband edge: 50.62 Hz
- Upper transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 50.88 Hz)
- Filter length: 1691 samples (6.605 s)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 50 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter ord

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s


Fitting ICA took 1.3s.
Applying ICA to Raw instance
    Transforming to ICA space (19 components)
    Zeroing out 1 ICA component
    Projecting back using 19 PCA components
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.
Not setting metadata
10 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 10 events and 2560 original time points ...
0 bad epochs dropped
Effective window size : 8.000 (s)
Averaging across epochs before plotting...


In [ ]:
# epochs_data = epochs.get_data()
# print("epochs_data.shape = (10段, 19通道, 256HZ) = ", epochs_data.shape)